In [ ]:
# toyexample of miRDM-rfGA with toydata

In [ ]:
%matplotlib inline

In [ ]:
# Load package
import pandas as pd
import numpy as np
import random

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from deap import creator, base, tools, algorithms
import sys

# random sampling
from random import sample

# combination
from itertools import combinations

In [ ]:
import warnings

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

#warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# parameter settings based on setting 5
random.seed(42)
_functional_relevance_lookup_dict = {}
_population = 1000
_generation = 300
_penalty_weight = 7
_num_one = 12
# _mu = 50
# _lambda = 100
_cxpb = 0.8
_mutpb = 0.003

# def trim_newline(s):
#     if s[-1] == '\n':
#         return s[0:-1]
    
#     return s

# workdir = "D://miR_Rank_Feautre_selection//GA_test"
# _functional_relevance_lookup_dict = {}
# with open(workdir + '//FR.txt', 'r') as fh:
#     fh.readline() # skip the first line
#     for line in fh:
#         elements = line.split('\t')
#         _functional_relevance_lookup_dict[elements[0]] = float(trim_newline(elements[1]))

# def functional_relevance_score(items):
#     sum_scores = 0
#     for x in items:
#         try:
#             sum_scores += _functional_relevance_lookup_dict[x]
#         except KeyError:
#             continue
    
#     return sum_scores


# Define fitness function
def average(l):
    """
    Returns the average between list elements
    """
    return (sum(l)/float(len(l)))


def evaluate(individual, X, y):
#     print (individual)
#     print ("Indivdial: num of 0: %d " % individual.count(0))
#     print ("Indivdial: num of 1: %d " % individual.count(1))
#     print ("The num of genes: %d" % len(individual))
    if(individual.count(0) != len(individual)):
        # 조건 1: 1의 개수가 5보다 클 때, penalty 값을 
        
        # get index with value 0
        cols = [i for i in range(len(individual)) if individual[i] == 1]

        
        #if len(individual) - len(cols) < 5:
        #    return 10000, 10000, 10000
        
        #if len(individual) - len(cols) > 20:
        #    return 10000, 10000, 10000
        
        # get features subset
#         X_parsed = X.drop(X.columns[cols], axis=1)
#         X_subset = pd.get_dummies(X_parsed)

        # apply classification algorithm
        clf = RandomForestClassifier()
        #clf = LogisticRegression()
        
        # fr_score = functional_relevance_score(X_parsed.columns.to_list())
 
        avg_roc_score = 100 * average(cross_val_score(clf, X.iloc[:,cols], y, cv=3, scoring='roc_auc'))
        penalty_function = _penalty_weight * abs(individual.count(1) - 3)
#         print("Avg roc score: " + str(avg_roc_score) + " Penalty weight: " + str(penalty_function) + " len 1: " + str(individual.count(1)))
        # return average(cross_val_score(clf, X_subset, y, cv=5), scoring = "roc_auc") - _penalty_weight * abs(individual.count(1) - 3), 1/fr_score #, (len(individual) - len(cols)) / len(individual)
        return avg_roc_score - penalty_function,
#         shuffle = cross_validation.KFold(len(X), n_folds=3, shuffle=True)
#         scores = cross_val_score(rf, X, y, cv=shuffle, scoring='roc_auc')
    else:
        # return 10000, 10000 #, 10000
        return -100000,

In [ ]:
# data path
workdir = "./toydata" # toydata path
# toy miRNA-Seq data loading
df_genomic = pd.read_csv(workdir + "//toydata.csv"
                    , index_col = ["id"])
y = df_genomic["Class"] #Class
#X_DN = df_e1.drop(["Class", 'hsa-let-7b-5p', 'hsa-miR-23b-3p', 'hsa-let-7c-5p', 'hsa-miR-30a-3p', 'hsa-miR-200c-3p'], axis = 1) #Rest dataseet
#X_DN = df_e1.drop(["Class", 'hsa-miR-1290'], axis = 1) #Rest dataseet
#X_DN = df_e1.drop(["Class", 'hsa-miR-642a-5p', 'hsa-miR-1343-3p', 'hsa-miR-642a-3p', 'hsa-miR-99b-3p', 'hsa-miR-671-3p', 'hsa-miR-3144-5p', 'hsa-miR-320c', 'hsa-miR-150-5p'], axis = 1) #Rest dataseet
#X_DN = df_e1.drop(["Class", 'hsa-miR-642a-5p', 'hsa-miR-642a-3p', 'hsa-miR-99b-3p', 'hsa-miR-891a-5p', 'hsa-miR-320c', 'hsa-miR-150-5p'], axis = 1) #Rest dataseet
df_genomic = df_genomic.drop(["Class"], axis = 1) #Rest dataseet
# split data train 70 % and test 30 %
#drop and rotate dataset
df_genomic = df_genomic.T[(df_genomic != 0).any()].T

_number_of_samples = len(df_genomic)
_number_of_items = len(df_genomic.columns.to_list())


In [ ]:
len(df_genomic)

In [ ]:
# remove sparse miRNAs
cut_off = 0.9
for x in df_genomic.columns:
    try:
        num_zero = df_genomic[x].value_counts(0)[0]
        if num_zero / _number_of_samples > cut_off:
            df_genomic = df_genomic.drop([x], axis = 1)
    except KeyError:
        continue

In [ ]:
df_genomic

In [ ]:
#get data excluding labels
X = df_genomic.iloc[:,]

# z-normalization
X = (X - X.mean())/X.std()

In [ ]:
# GA toolbox setup
creator.create("MyFitnessMulti", base.Fitness, weights=(1.0,)) # Note here <- I used only two weights!  (at first, I tried weights=(-1.0 , -1.0, 1.0)) but it crashes. With deap, you cannot do such a thing.
creator.create("Individual", list, fitness=creator.MyFitnessMulti)

toolbox = base.Toolbox()

#def f(a, b, c): 
#    return np.random.choice(np.arange(a, b), p=c)
  
# A partial function that calls f with 
# a as 3, b as 1 and c as 4. 
#g = partial(f, 0, 2) 




# Attribute generator
def random_individual_three():
    zero_list = [0 for i in range(len(X.columns))]
    index_for_one = np.random.choice(len(X.columns), _num_one, replace=True)
#     index_for_one = random.sample(range(len(X.columns)),_num_one) # 중복을 허용하지 않는 index random sampling
    # index_for_one = list(combinations(range(len(X.columns)),_num_one)) # combination 생성
    for i in index_for_one:
        zero_list[i]= 1 
    
#     print(zero_list.count(1))
    return zero_list
    

# Structure initializers
toolbox.register("rand_ind",random_individual_three)
toolbox.register("individual", tools.initIterate, creator.Individual
                 ,toolbox.rand_ind) #, n=30) #
# toolbox.register("individual", tools.initIterate, creator.Individual, random.sample(range(len(X.columns)),_num_one))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Registration of crossover & mutation functions
#toolbox.register("mate", tools.cxUniform, indpb=0.8)
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=_mutpb)

# Registration of selection function
toolbox.register("select", tools.selBest)
# toolbox.register("select", tools.selRoulette)
# toolbox.register("select", tools.selNSGA2) # NSGA-2 applies to multi-objective problems such as knapsack problem
# toolbox.register("select", tools.selTournament, k=1, tournsize=3)

toolbox.register("evaluate", evaluate, X=X, y=y)
logbook = tools.Logbook()

In [ ]:
len(X.columns)

In [ ]:
# from functools import partial
# random.seed(42)
# gen_idx = random.sample(range(len(X.columns)), _num_one)
# print(gen_idx)
# print(list(gen_idx))

In [ ]:
pop = toolbox.population(n=_population)
hof = tools.HallOfFame(1) # a ParetoFront may be used to retrieve the best non dominated individuals of the evolution
# hof = tools.ParetoFront() # a ParetoFront may be used to retrieve the best non dominated individuals of the evolution

# assign statistics for fitness function in each population
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean, axis=0)
stats.register("std", np.std, axis=0)
stats.register("min", np.min, axis=0)
stats.register("max", np.max, axis=0)

# pop, log = algorithms.eaMuPlusLambda(pop, toolbox, _mu, _lambda, _cxpb, _mutpb, 
#                                      ngen=_generation, stats=stats, halloffame=hof) #, verbose=True)

# pop, log = algorithms.eaSimple(pop, toolbox, cxpb=_cxpb, mutpb=_mutpb, ngen=_generation, 
#                                   stats=stats, halloffame=hof, verbose=True)

# to check individuals in each popluation
population = []

# Run GA
for i in range(0,_generation + 1):
    print ("Generation: %d" % i)
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=_cxpb, mutpb=_mutpb, ngen=1, stats=stats, halloffame=hof, verbose=True)
    population.append(pop)
# pop, log = algorithms.eaSimple(pop, toolbox, cxpb=_cxpb, mutpb=_mutpb, ngen=_generation, stats=stats, halloffame=hof, verbose=True)

#for ind, fit in zip (pop, )

In [ ]:
print("Individulas in the hall of fame population")
for x in hof:
    print(x)

In [ ]:
print("The # of individuals in the hof population: %d " % len(hof))

In [ ]:
# Selection of the best individual
_individual = tools.selBest(hof, 1)[0]
# _individual = tools.selNSGA2(hof, 1)[0]
_individual_features = [X.columns[i] for i in range(len(_individual)) if _individual[i] == 1]
print("The best individual is :" + str(_individual))

In [ ]:
print("The best features are: \t" + str(_individual_features))

In [ ]:
# Fitness score of best individual
print('Fitness score: \t' + str(_individual.fitness))



In [ ]:
print('Number of Features in Subset: \t' + str(_individual.count(1)))


In [ ]:
print('Individual: \t' + str(_individual))

In [ ]:
print('Feature Subset\t: ' + str(_individual_features))

In [ ]:
_individual.fitness

In [ ]:
len(_individual_features)

In [ ]:
for x in _individual_features:
    print ("'{0}', ".format(x), end='')

In [ ]:
_individual.count(1)